In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import h5py as h5
import numpy as np
import dask
import dask.array as da
from PIL import Image
import glob

# Outils informatiques pour le Big Data

## Examen du 10 décembre 2019

- **Prénom:**
- **Nom:**

*Attention vous devez déposer ce fichier sur [cursus](https://cursus.univ-rennes2.fr/course/view.php?id=11467) avant 12h00*

Le barême est sur 25 points mais la note maximale sera 20

Le code SeLaLib (Semi-Lagrangian Library) permet de résoudre numériquement les équations d'évolution d'un plasma dans un processus de fusion magnétique. Lors de son éxécution ce code produit des fichiers permettant la visualisation de différentes quantités physiques. Une simulation a produit 1000 fichiers archivés dans le fichier `fvalues.tgz`

### Exercice 1 (2 points)

Ecrire le programme python permettant d'extraire ces fichiers contenus dans le fichier `fvalues.tgz` dans un répértoire nommé `data`.

In [ ]:
import os  # library to get directory and file paths
import tarfile # this module makes possible to read and write tar archives

def extract_data(src, dest):
   #
   #
         
extract_data('fvalues','data') 

### Exercice 2 (1 point)

Les valeurs contenues dans ces fichiers représentent la fonction de distribution des électrons dans un espace $(r,\theta)$. Ce champ stocké dans les fichiers au format hdf5
présents dans le répertoire data sont de la forme
"f000*-values.h5". Créer la liste python `filenames` contenant les chemins vers chacun de ces fichiers.

In [ ]:
filenames = #...

### Exercice 3 ( 2 points)

Pour lire ces fichiers au format hdf5 et récupérer les données qui nous intéressent, nous allons utiliser le package [h5py](http://docs.h5py.org/en/stable/)

In [ ]:
f = h5.File("data/f0499-values.h5", "r")

L'objet f possède les mêmes caractéristiques qu'un dictionnaire python. Pour afficher les noms des datasets contenus dans `f`, il suffit d'observer les valeurs retournées par `f.keys()`

In [ ]:
list(f.keys())

 La valeurs de $f(r,\theta,t)$ sont contenues dans le dataset nommé `values`. Ces valeurs peuvent être obtenues avec la fonction `f.get("values")`

In [ ]:
f.get("values")

En vous inspirant de la démarche décrite ci dessus, récupérer les valeurs des datasets `x` et `y` du fichier polar-mesh.h5. 

In [ ]:
#
#
#

### Exercice 4 (2 points)

Pour tracer le contour de $f(r,\theta)$, utilisons matplotlib:

In [ ]:
z = f.get("values")
plt.contourf(x, y, z)
plt.axis('equal')
plt.tight_layout()
plt.axis('off');

On peut sauvegarder cette image avec les instructions:
```py
plt.savefig("data/f0499-values.png")
plt.close() # plot is not displayed on notebook
```

Ecrire une fonction `save_plot` avec comme argument le nom du fichier HDF5. Cette fonction permet de créer le fichier png correspondant. 

In [ ]:
def save_plot( fn ):
    #
    #
    #

Utiliser une boucle pour créer tous les fichiers png.
Paralléliser cette boucle en utilisant `concurrent.futures`. Calculer l'efficacité de la parallélisation en divisant temps séquentiel par le temps parallèle multiplié par le nombre de processeurs $p$. 
$$
\mbox{efficiency} = \frac{ T_1 }{ p T_p} \times 100
$$

Une valeur de 100 % correspond à une efficacité parfaite.

In [ ]:
from tqdm.notebook import tqdm

for fn in tqdm(filenames):
    save_plot(fn)

In [ ]:
import concurrent.futures 

#
#

In [ ]:
efficiency = #...

### Exercise 5 (4 points)

Ecrire une fonction `scale` permettant de normaliser les données contenues dans un fichier.  L'objectif est de créer une image avec des niveaux de gris. Les valeurs devront etre de type `numpy.uint8`
et comprises entre 0 et 255.

```py
X = np.random.random((4,4))
# array([[0.5445994 , 0.69475429, 0.24044862, 0.62431428],
#        [0.15061212, 0.70055941, 0.2439314 , 0.51174016],
#        [0.93747823, 0.71917128, 0.9646407 , 0.22400018],
#        [0.4161566 , 0.26919381, 0.26040576, 0.55806422]])
scale(X)
# array([[ 90, 146,   0, 218],
#        [ 97, 220,  46, 195],
#        [ 12,  89, 255,   1],
#        [218, 170, 185,  24]], dtype=uint8)
```

In [ ]:
def scale(x) :
    #...

Le tableau obtenu permet de tracer une image avec le package python [pillow](https://pillow.readthedocs.io/en/stable/). Exécutez la cellule ci-dessous pour
voir un exemple.

In [ ]:
from PIL import Image

# Tableau 2D avec des entiers entre 0 et 255
X = np.arange(256).astype(np.uint8) 
XY = X * X[:,np.newaxis]              

Image.fromarray(XY)

Ecrire une fonction `plot_image`, avec comme argument le nom du fichier
qui permet d'afficher image produite par pillow.

In [ ]:
def plot_image( fn ):
    #...
    #...
    
plot_image(filenames[499])

le package `pillow` est intéressant car beaucoup plus rapide que matplotlib. Il permet par exemple de parcourir les images à l'aide d'un `widget`.

In [ ]:
from tqdm.notebook import tqdm
from PIL import Image

def create_image(fn):
    with h5.File(fn, "r") as f:
        dataset = "values"
        Z = f.get(dataset)
        return scale(Z)

def create_frames(filenames):

    return [create_image(fn) for fn in tqdm(filenames)]
    
frames = create_frames(filenames)

In [ ]:
from ipywidgets import interact, IntSlider

interact(lambda i: Image.fromarray(frames[i]), 
         i = IntSlider(min=0,
                    max=len(frames)-1,
                    step=1,
                    value=0, 
                    continuous_update=True))

### Exercice 6 (6 points)

Pour faciliter le traitement des données nous allons stocker l'ensemble des valeurs dans
un `dask.array` à trois dimensions.
- La première dimension correspond au numéro du fichier
- La deuxième et la troisieme sont les dimensions des tableaux contenus dans chacun des fichiers.
écrire le programme Python permettant de créer ce `dask.array` nommé `dask_frames`
Pour créer cette variable utiliser la fonction [stack](https://docs.dask.org/en/latest/array-creation.html)

In [ ]:
values

In [ ]:
display_frame(values[500,:,:].compute())

### Exercice 7 ( 8 points)

Les données proviennent de [UCI Machine Learning Repository Combined Cycle Power Plant Data Set](https://archive.ics.uci.edu/ml/datasets/Combined+Cycle+Power+Plant).
Il s'agit de données liées au fonctionnement d'une centrale à gaz en fonction des conditions atmosphériques.

**Variables**
- AT = Atmospheric Temperature in C
- V = Exhaust Vaccum Speed
- AP = Atmospheric Pressure
- RH = Relative Humidity
- PE = Power Output

L'objectif de cet exercice est de calculer les corrélations entre ces différentes variables.
Les observations sont contenues dans 5 feuilles excel du fichier "Folds5x2_pp.xlsx"
- Lire le fichier à l'aide de la fonction pandas [read_excel](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_excel.html). Quel est le type de la variable renvoyée par la fonction ?
- Ecrire une fonction `select` permettant de regrouper toutes les observations d'une variable dans une série pandas. La fonction `pandas.concat` peut vous aider.
- Utiliser la fonction `select` et la méthode `corr` pour calculer les deux variables les plus corrélées.
- Paralléliser cette boucle à l'aide de `concurrent.futures`. Les données étant peu volumineuses avec seulement cinq variables, la parallélisation n'apporte pas de gain particulier.